In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import os
import shutil
import pandas as pd
import json

In [2]:
zomato=pd.read_csv('Zomato.csv')

In [3]:
zomato

,Unnamed: 0,Links,Name,Rating,Cuisines,Reviews,Avg waiting time,Cost / Person
0,0,https://www.zomato.com/nagpur/haldirams-hotspo...,Haldiram's Hotspot,4.2,"['Street Food', ' Mithai', ' North Indian', ' ...",21.8K,28,150
1,1,https://www.zomato.com/nagpur/faasos-sadar/order,Faasos,3.7,"['Fast Food', ' North Indian', ' Biryani', ' R...",7641,37,200
2,2,https://www.zomato.com/nagpur/khichadiwala-ram...,Khichadiwala,4.1,"['Healthy Food', ' North Indian', ' Beverages']",10.3K,31,100
3,3,https://www.zomato.com/nagpur/checkers-1-sadar...,Checkers,4.0,"['Chinese', ' Continental', ' Fast Food', ' No...",2557,29,200
4,4,https://www.zomato.com/nagpur/sardaarji-ramdas...,SardaarJi,4.1,"['Fast Food', ' North Indian', ' Rolls', ' Mug...",13.1K,30,250
...,...,...,...,...,...,...,...,...
1291,1291,https://www.zomato.com/nagpur/shahdara-biryani...,Shahdara Biryani,4.0,"['North Indian', ' Biryani']",113,-99,200
1292,1292,https://www.zomato.com/nagpur/gawrani-tadka-wa...,Gawrani Tadka,3.9,"['North Indian', ' Maharashtrian']",33,-99,150
1293,1293,https://www.zomato.com/nagpur/ajmera-biryani-r...,Ajmera Biryani,3.9,['Biryani'],255,-99,250
1294,1294,https://www.zomato.com/nagpur/hyderabadi-birya...,Hyderabadi Biryani,3.6,"['Biryani', ' North Indian', ' Chinese', ' Rol...",1093,28,200


In [4]:
headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [ ]:
resmenu=[]
m1=[]
m2=[]
must_try_final=[]
chef_sp_final=[]
links=[]
for i in range(len(zomato)):
    veg = []
    print(zomato['Links'][i])
    try:
        link=zomato['Links'][i]
        links.append(link)
        linkpage=requests.get(link, headers=headers)
        #print(i,".",link)
        soup = BeautifulSoup(linkpage.text, 'lxml')
        sideCol=soup.find_all('p',class_='sc-1hez2tp-0')
        #print(sideCol)
        dc={}
        for i in sideCol:
            val = ""
            if '(' in i.text:
                s1 = len(i.text)
                s=i.text
                for x in range(s1-2,0,-1):
                    if (s[x] == '('):
                        break
                    val = s[x] + val
                dc[((s.split('('))[0]).rstrip()]=int(val)
        resmenu.append(dc)
        #print(dc)
        count=0
        menuVEG={}
        menuNONVEG = {}
        menuL=soup.find_all('div',class_='sc-1s0saks-17')
        num = 0
        l = {}
        for k in dc:
            #print(k,dc[k])
            key=[]
            veg = {}
            nonveg = {}
            for j in range(dc[k]):
                #print(count)
                #print(menuL[count].text)
                #print(veg)
                key = []
                item_name = menuL[count].find('h4',class_ ='sc-1s0saks-15 iSmBPS')
                must_try = menuL[count].find('div',class_ ='sc-2gamf4-0 cRxPpO')
                chef_sp = menuL[count].find('div',class_ ='sc-2gamf4-0 fQRUpA')
                VEG = menuL[count].find('div',class_='sc-1tx3445-1')
                if (must_try != None):
                    key.append(1)
                else:
                    key.append(0)
                if (chef_sp != None):
                    key.append(1)
                else:
                    key.append(0)
                if (k == 'Recommended'):
                    key.append(1)
                    l[item_name.text] = key
                else:
                    if (VEG['type'] == 'veg' and item_name.text in l.keys()):
                        veg[item_name.text] = l[item_name.text]
                    elif (VEG['type'] == 'veg'):
                        key.append(0)
                        veg[item_name.text] = key
                    if (VEG['type'] == 'non-veg' and item_name.text in l.keys()):
                        nonveg[item_name.text] = l[item_name.text]
                    elif (VEG['type'] == 'non-veg'):
                        key.append(0)
                        nonveg[item_name.text] = key
                    if (VEG['type'] == 'egg' and item_name.text in l.keys()):
                        nonveg[item_name.text] = l[item_name.text]
                    elif(VEG['type'] == 'egg'):
                        key.append(0)
                        nonveg[item_name.text] = key
                count+=1
            menuVEG[k] = veg
            menuNONVEG[k] = nonveg
        m1.append(menuVEG)
        m2.append(menuNONVEG)
        print(len(m1) , len(m2))
    except:
        m1.append({})
        m2.append({})
        print("ERROR"+ "="*100 )

https://www.zomato.com/nagpur/haldirams-hotspot-sadar/order
1 1
https://www.zomato.com/nagpur/faasos-sadar/order
2 2
https://www.zomato.com/nagpur/khichadiwala-ramdaspeth/order
3 3
https://www.zomato.com/nagpur/checkers-1-sadar/order
4 4
https://www.zomato.com/nagpur/sardaarji-ramdaspeth/order
5 5
https://www.zomato.com/nagpur/uttar-dakshin-by-naivedhyam-sadar/order
6 6
https://www.zomato.com/nagpur/tiranga-saoji-bhojanalay-ayodhya-nagar/order
7 7
https://www.zomato.com/nagpur/nanking-chinese-restaurant-civil-lines/order
8 8
https://www.zomato.com/nagpur/manus-missal-co-pratap-nagar/order
9 9
https://www.zomato.com/nagpur/babbus-food-plaza-sadar/order
10 10
https://www.zomato.com/nagpur/dinshaws-ice-cream-parlour-gandhibagh/order
11 11
https://www.zomato.com/nagpur/chaat-shaala-ravi-nagar/order
12 12
https://www.zomato.com/nagpur/cake-links-sadar/order
13 13
https://www.zomato.com/nagpur/jds-dessert-boutique-chaoni/order
14 14
https://www.zomato.com/nagpur/jay-ambika-bhojnalay-gandhiba

In [ ]:
m1

In [ ]:
m2

In [ ]:
df=pd.DataFrame({'Links':links,'Veg_menu':m1,'Non_veg_menu':m2,})

In [ ]:
df

In [ ]:
z=zomato.merge(df,how='inner',on='Links')

In [ ]:
z['Veg_menu'][0]

In [ ]:
z.to_json('data_Z.json')

In [ ]:
df1 = pd.read_json("data_Z.json")

In [ ]:
df1

In [ ]:
df1['Veg_menu'][0]['Thali']["Haldiram's Special Thali"]